In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import pandas as pd
from ortools.sat.python import cp_model

In [ ]:
from ortools.sat.python.cp_model import _SumArray

In [ ]:
class DataFrameSolutionAggregator(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__pd_variables = []
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        values = []
        for v in self.__variables:
            values.append(self.Value(v))
        self.__pd_variables.append(values)

    def get_data_frame(self):
        df = pd.DataFrame(self.__pd_variables, columns=self.__variables)
        return df.loc[:, (df != 0).any(axis=0)]

    def solution_count(self):
        return self.__solution_count

In [ ]:
with open('values.json', 'r') as file:
    values = json.load(file)

In [ ]:
with open('tech_tree.json', 'r') as file:
    techs = json.load(file)

In [ ]:
def addAllTechs(model, techs):
    tech_dict = {}
    for t1 in techs:
        k1 = t1['name']
        tech_dict[k1] = model.NewBoolVar(k1)
        for t2 in t1['allows']:
            k2 = t2['name']
            tech_dict[k2] = model.NewBoolVar(k2)
            for t3 in t2['allows']:
                k3 = t3['name']
                tech_dict[k3] = model.NewBoolVar(k3)
    return tech_dict
#t_d = addAllTechs(cp_model.CpModel(), techs)

In [ ]:
def buildTechTreeConstraints(model, techs, tech_dict):
    for t1 in techs:
        k1 = t1['name']
        for t2 in t1['allows']:
            k2 = t2['name']
            model.Add(tech_dict[k2] == 0).OnlyEnforceIf(tech_dict[k1].Not())
            for t3 in t2['allows']:
                k3 = t3['name']
                model.Add(tech_dict[k3] == 0).OnlyEnforceIf(tech_dict[k2].Not())

In [ ]:
def buildTechUnitsConstraints(model, techs, tech_dict, sym_dict):
    for t1 in techs:
        k1 = t1['name']
        for u in t1['units']:
            model.Add(sym_dict[u] == 0).OnlyEnforceIf(tech_dict[k1].Not())
        for t2 in t1['allows']:
            k2 = t2['name']
            for u in t2['units']:
                model.Add(sym_dict[u] == 0).OnlyEnforceIf(tech_dict[k2].Not())
            for t3 in t2['allows']:
                k3 = t3['name']
                for u in t3['units']:
                    model.Add(sym_dict[u] == 0).OnlyEnforceIf(tech_dict[k3].Not())

In [ ]:
def linkTechAndTech(model, techs, tech_dict, sym_dict):
    model.Add(sym_dict['tier-1'] == _SumArray([tech_dict[t['name']] for t in techs]))
    model.Add(sym_dict['tier-2'] == _SumArray([tech_dict[t['name']] for u in techs for t in u['allows']]))
    model.Add(sym_dict['tier-2'] == _SumArray([tech_dict[t['name']] for v in techs for u in v['allows'] for t in u['allows']]))

In [ ]:
def addAllVars(model, values):
    eq_dict = {}
    sym_dict = {}
    for category in values.keys():
        for element, score in values[category].items():
            # replace 10 with max value of elements, static or dynamic
            sym_dict[element] = model.NewIntVar(0, 10, element) 
            eq_dict[element] = score * sym_dict[element]
    return eq_dict, sym_dict

In [ ]:
def buildFullScoreConstraint(model, eq_dict, player_score):
    cat_eq_dict = {}
    for category in values.keys():
        cat_eq_dict[category] = _SumArray([eq_dict[element] for element in values[category].keys()])
    model.Add(_SumArray([cat_eq_dict[cat] for cat in cat_eq_dict.keys()]) == player_score)

In [ ]:
def buildVisionConstraint(model, eq_dict, vision_score):
    model.Add(eq_dict['revealed'] == vision_score)

In [ ]:
def solve(raw_score, full_score):
    model = cp_model.CpModel()
    eq_dict, sym_dict = addAllVars(model, values)

    buildScoreConstraint(model, eq_dict, full_score, True)
    buildScoreConstraint(model, eq_dict, raw_score, False)
    model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)


    solver = cp_model.CpSolver()
    solution_printer = DataFrameSolutionAggregator(sym_dict.values())
    status = solver.SearchForAllSolutions(model, solution_printer)
    return solution_printer

In [ ]:
# superb_ruin
scores = [
    ([515, 515], []),
    ([565, 565], [440, 440]),
    ([590, 600], [450, 450]),
    ([855, 635], [700, 460]),
    ([1385, 670], [1230, 460]),
    ([1860, 1215], [1700, 990]),
    ([2230, 1815], [2010, 1530])
]

In [ ]:
player_scores = [
    [565, 440],
    [590, 450],
    [855, 700],
    [1385, 1230],
    [1860, 1700],
    [2230, 2010]
]

In [ ]:
player_2_scores = [
    [565, 440],
    [600, 450],
    [635, 460],
    [670, 460],
    [1215, 990],
    [1815, 1530]
]

In [ ]:
def divide_scores(scores):


    return selected_scores + divide_score(scores[1:])

In [ ]:
for i in range(len(player_scores) - 1):
    raw_delta = player_scores[i + 1][1] - player_scores[i][1]
    full_delta = player_scores[i + 1][0] - player_scores[i][0]
    print(full_delta, raw_delta)
    # sp = solve(raw_score=raw_delta, full_score=full_delta)
    # print(sp.get_data_frame().to_string())

In [ ]:
print(sp.get_data_frame())

# Equations: 

full score = score(units) + score(tech) + score(city) + score(vision)

raw score =  score(units) + score(tech) + score(city)

'remaining stars + spt + extra stars = stars(units) + stars(tech) + star(building)'

In [ ]:
model = cp_model.CpModel()
eq_dict, sym_dict = addAllVars(model, values)
tech_dict = addAllTechs(model, techs)

full_delta = 25
raw_delta = 10
buildVisionConstraint(model, eq_dict, full_delta - raw_delta)
buildFullScoreConstraint(model, eq_dict, full_delta)
model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)
buildTechUnitsConstraints(model, techs, tech_dict, sym_dict)
buildTechTreeConstraints(model, techs, tech_dict)
linkTechAndTech(model, techs, tech_dict, sym_dict)

solver = cp_model.CpSolver()
solution_printer = DataFrameSolutionAggregator(list(sym_dict.values()) + list(tech_dict.values()))
status = solver.SearchForAllSolutions(model, solution_printer)


In [ ]:
map(lambda x: x.getElement(), tech_dict.values)

In [ ]:
x = solution_printer.get_data_frame()
x.drop_duplicates()

In [ ]:
x = solution_printer.get_data_frame()
x

9*20 (territories)+ 100 (level 1 city)+ tech+ units= raw score turn 0

turn 0, vision score is always 125=25*5